The RAG triad is made up of 3 evaluations: context relevance, groundedness and answer relevance. Satisfactory evaluations on each provides us confidence that our LLM app is free from hallucination.

In [54]:
import utils

import os
import openai
openai.api_key = ""

In [17]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [18]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [19]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [20]:
from llama_index.llms.openai import OpenAI
from local_utils import build_sentence_window_index

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)


In [21]:
from local_utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [22]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

Start by selecting a project that interests you within the field of AI. Consider beginning with a smaller project to gain experience and learn from the process. Clearly define the goals and objectives of your project, and communicate the value of what you hope to achieve. Engage with colleagues, mentors, or managers to gather feedback and refine your ideas. As you progress, focus on building a portfolio of projects that demonstrate your skill progression from simple to more complex undertakings. Remember that communication is key in explaining your thinking and accomplishments to others, which can help you gain support and resources for larger projects.


In [23]:
import nest_asyncio

nest_asyncio.apply()

In [24]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

1. Answer Relevance

In [25]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


2. Context Relevance

In [26]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [40]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


3. Groundedness

In [41]:
from trulens.feedback.v2.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [42]:
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


Evaluation of the RAG application

In [43]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'abc

In [44]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [45]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [46]:
eval_questions.append("How can I be successful in AI?")

In [47]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

calling <function BaseQueryEngine.query at 0x152fcb880> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x11d8d5570>, 'What are the keys to building a career in AI?')
calling <function RetrieverQueryEngine.retrieve at 0x153ad0820> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x11d8d5570>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function BaseRetriever.retrieve at 0x1533aeb00> with (<llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever object at 0x1603cb910>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function VectorIndexRetriever._retrieve at 0x153a08d30> with (<llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever object at 0x1603cb910>, QueryBundle

/opt/anaconda3/lib/python3.10/site-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


calling <function BaseQueryEngine.query at 0x152fcb880> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x11d8d5570>, 'How can altruism be beneficial in building a career?')
calling <function RetrieverQueryEngine.retrieve at 0x153ad0820> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x11d8d5570>, QueryBundle(query_str='How can altruism be beneficial in building a career?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function BaseRetriever.retrieve at 0x1533aeb00> with (<llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever object at 0x1603cb910>, QueryBundle(query_str='How can altruism be beneficial in building a career?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function VectorIndexRetriever._retrieve at 0x153a08d30> with (<llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever object at 0x16

In [48]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

app_id  \
0  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
1  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
2  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
3  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
4  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  app_json  \
0  {'tru_class_info': {'name': 'TruLlama', 'module': {'package_name': 'trulens.apps.llamaindex', 'module_name': 'trulens.apps.llamaindex.tru_llama'}, 'bases': [{'name': 'TruLlama', 'module': {'package_name': 'trulens.apps.llamaindex', 'module_name': 'trulens.apps.llamaindex.tru_llama'}, 'bases': None}, {'name': 'App', 'module': {'package_name': 'trulens.core', 'module_name': 'trulens.core.app'}, 'bases': None}, {'name': 'AppDefinition', 'module': {'package_name': 'trulens.core.schema', 'module_name': 'trulens.core.schema.app'}, 'bases': None}, {'name': 'WithClassInfo', 'module': {'package_name': 'trulens.core.utils', 'module_name': 'trulens.core.utils.pyschema'}, 'bases': None}, {'name': 'SerialModel', 'module': {'package_name': 'trulens.core.utils', 'module_name': 'trulens.core.utils.serial'}, 'bases': None}, {'name': 'BaseModel', 'module': {'package_name': 'pydantic', 'module_name': 'pydantic.main'}, 'bases': None}, {'name': 'WithInstrumentCallbacks', 'module': {'package_name': 'trulens.core', 'module_name': 'trulens.core.instruments'}, 'bases': None}, {'name': 'Hashable', 'module': {'package_name': 'collections', 'module_name': 'collections.abc'}, 'bases': None}, {'name': 'Generic', 'module': {'package_name': '', 'module_name': 'typing'}, 'bases': None}, {'name': 'object', 'module': {'package_name': '', 'module_name': 'builtins'}, 'bases': None}]}, 'app_id':

In [49]:
print(records.columns)


Index(['app_id', 'app_json', 'type', 'record_id', 'input', 'output', 'tags',
       'record_json', 'cost_json', 'perf_json', 'ts', 'Answer Relevance',
       'Answer Relevance_calls', 'Answer Relevance feedback cost in USD',
       'Context Relevance', 'Context Relevance_calls',
       'Context Relevance feedback cost in USD', 'Groundedness',
       'Groundedness_calls', 'Groundedness feedback cost in USD', 'app_name',
       'app_version', 'latency', 'total_tokens', 'total_cost',
       'cost_currency'],
      dtype='object')


In [52]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Context Relevance,Answer Relevance
0,How can I be successful in AI?,"To be successful in AI, it is important to focus on building a strong professional network or community, as teamwork and collaboration are key in this field. Additionally, developing good habits in various aspects of life such as learning, work, and personal relationships is crucial for long-term success. Embracing altruism and aiming to lift others during your own journey can lead to better outcomes. Overcoming challenges like imposter syndrome and being open to learning from mistakes are also essential in progressing in AI. Finally, building a portfolio of projects that show skill progression, effective communication, and leadership skills are vital for success in the field of AI.",NaN,NaN,NaN
1,Is it normal to find parts of AI challenging?,It is normal to find parts of AI challenging.,NaN,NaN,NaN
2,What are some common challenges in AI?,"Common challenges in AI include transitioning roles or industries, building a portfolio of projects that demonstrate skill progression, effectively communicating the value of your work, and developing leadership skills for larger projects.",NaN,NaN,NaN
3,What is the first step to becoming good at AI?,Learning foundational technical skills is the first step to becoming good at AI.,NaN,NaN,1.000000
4,Who are some accomplished individuals who have experienced imposter syndrome?,Individuals who have experienced imposter syndrome include those who have published seminal AI papers and have struggled with technical challenges at some point in their careers.,NaN,0.333333,0.333333
5,What is imposter syndrome and how does it relate to AI?,"Imposter syndrome is a feeling of inadequacy or self-doubt despite evident success or accomplishments. In the context of AI, imposter syndrome can arise when individuals feel like they do not belong in the AI community due to perceived lack of knowledge or skills, even if they have made progress or achieved success in the field. This feeling can be common among those learning AI, as the field is technically complex and can involve challenges that may make individuals doubt their abilities. However, it is important to recognize that overcoming imposter syndrome is a shared experience among many in the AI community, and it should not deter individuals from pursuing a career or involvement in AI.",NaN,0.500000,1.000000
6,How can altruism be beneficial in building a career?,Helping others during each step of your own career journey can lead to better outcomes for yourself.,0.500000,0.500000,0.666667
7,What are some good habits to develop for a successful career?,"Developing good habits in eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while staying healthy.",0.500000,0.833333,0.333333
8,What is the importance of networking in AI?,"Networking is crucial in AI as it helps individuals build a strong professional network that can provide support, advice, and opportunities when needed. It is emphasized that having a community of like-minded individuals can be more beneficial than traditional networking, especially for introverts who may find networking events challenging. Building relationships and connections within the AI community can lead to collaborations, knowledge sharing, and personal growth, ultimately propelling one's career forward.",0.500000,0.500000,1.000000
9,How can teamwork contribute to success in AI?,"Teamwork can contribute to success in AI by enabling collaboration, influence, and the exchange of ideas among team members. The ability to work effectively in teams allows individuals to leverage diverse perspectives, skills, and expertise to tackle complex AI projects more efficiently. Interpersonal and communication skills play a crucial role in fostering a collaborative environment where team members can work together, influence each other positively, and collectively achieve bet

In [53]:
tru.get_leaderboard(app_ids=[])

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
App_1,base,0.772727,0.681818,0.637626,2.015029,0.003184


In [51]:
tru.run_dashboard()

Starting dashboard ...
Dashboard already running at path:   Network URL: http://192.168.1.134:60517



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>